In [19]:
import os
import re
import glob
import os
import sys
import ast
import os.path as op
from collections import defaultdict
from copy import deepcopy
import copy

import pickle
import mne
import scipy
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import set_config
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.feature_selection import SelectKBest, SelectPercentile, f_regression, SelectFpr, SelectFdr, SequentialFeatureSelector
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn import datasets, linear_model

warnings.filterwarnings("ignore")

sns.set_theme(style="whitegrid", palette="deep")


In [6]:
#create dataframe from participants.tsv containing columns 1 and 2
df = pd.read_csv('../ds003944/participants.tsv', sep='\t', usecols=[0,1])
print(df)


   participant_id       type
0        sub-1448    Control
1        sub-1824  Psychosis
2        sub-1971    Control
3        sub-1983  Psychosis
4        sub-1989    Control
..            ...        ...
77      sub-2184A  Psychosis
78      sub-2193A    Control
79      sub-2214A    Control
80      sub-2217A  Psychosis
81      sub-2221A    Control

[82 rows x 2 columns]


In [7]:
#add mne object to dataframe from vmrk files within folder that name equals value in column 1
df['psyhosis_raw_data'] = [mne.io.read_raw_brainvision('../ds003944/'+str(i)+'/eeg/'+str(i)+'_task-Rest_eeg.vhdr', preload=True) for i in df['participant_id']]

Extracting parameters from ../ds003944/sub-1448/eeg/sub-1448_task-Rest_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 305999  =      0.000 ...   305.999 secs...
Extracting parameters from ../ds003944/sub-1824/eeg/sub-1824_task-Rest_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 300999  =      0.000 ...   300.999 secs...
Extracting parameters from ../ds003944/sub-1971/eeg/sub-1971_task-Rest_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 300999  =      0.000 ...   300.999 secs...
Extracting parameters from ../ds003944/sub-1983/eeg/sub-1983_task-Rest_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 300999  =      0.000 ...   300.999 secs...
Extracting parameters from ../ds003944/sub-1989/eeg/sub-1989_task-Rest_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 309999  =      0.000 ...   309.999 secs...
Extracting parameters from ../ds003944/sub-1990/eeg/sub-1990_task-Rest_eeg.vhdr...
Setting channel info structure...
Reading 0 ...

In [8]:
dir_with_data = "preprocessed-data/"
# df.insert(3, "processed_frequencies", False)

FREQ_BANDS = {    "theta": [4.5, 8.5],
                  "alpha": [8.5, 11.5],}


def eeg_power_band(epochs, FREQ_BANDS):
    """EEG relative power band feature extraction.

    This function takes an ``mne.Epochs`` object and creates EEG features based
    on relative power in specific frequency bands that are compatible with
    scikit-learn.

    Parameters
    ----------
    epochs : Epochs
        The data.

    Returns
    -------
    X : numpy array of shape [n_samples, 5]
        Transformed data.
    """

    spectrum = epochs.compute_psd(picks='eeg', fmin=0.5, fmax=30.)
    psds, freqs = spectrum.get_data(return_freqs=True)
    # Normalize the PSDs
    psds /= np.sum(psds, axis=-1, keepdims=True)

    X = []
    for fmin, fmax in FREQ_BANDS.values():
        psds_band = psds[:, (freqs >= fmin) & (freqs < fmax)].mean(axis=-1)
        X.append(psds_band.reshape(len(psds), -1))

    return np.concatenate(X, axis=1)

    
''' uwaga! poniżej jest try i except, żeby nie zatrzymywało się na błędzie, że nie ma pliku dla jakiegoś uczestnika (części plików nie ma, dla tych uczestników dla których był za bardzo zaszumiony sygnał'''
''' żeby przetestować czy wszystko działa i nie wywala błędów już na etapie pierwszegu uczestnika trzeba zakomentować to try i except'''

for sub_id in df['participant_id'].values:
    try:
        mat_epochs_readed = mne.read_epochs(f"{dir_with_data}{sub_id}-epo.fif")
        evoked = mat_epochs_readed.average()
        evoked_channels = evoked.pick_channels(['F7', 'F5', 'F3', 'F1', 'F2', 'F4', 'F6', 'F8', 'P7', 'P5', 'P3', 'P1', 'P2', 'P4', 'P6', 'P8', 'O1', 'O2', 'C5', 'C3', 'C1', 'C2', 'C4', 'C6', 'T9', 'T7', 'T8', 'T10'])
        print(eeg_power_band(evoked_channels, FREQ_BANDS))
        ''' tak jak widać liczy się to co powinno i printuje na ekran, ale nie umiem tego zapisać do data frame'a - powinno iść do nowej kolumny, w tym wierszu dla odpowiedniego uczestnika (aktualne sub_id)'''
        '''poniżej moje nieudolne próby zrobienia tego, które wywalały błąd'''
        # df.loc[df['participant_id'] == sub_id, 'processed_frequencies'] = eeg_power_band(evoked_channels, FREQ_BANDS)
        #df['processed_frequencies'] = df['participant_id'].where(df['participant_id']==sub_id, eeg_power_band(evoked_channels, FREQ_BANDS))
    except:
        print("file for "+sub_id+" doesn't exist")
        
    

Reading d:\Nina\Dokumenty\Studia\Kogni\magisterka\I semestr\machine learning\eggML-project\preprocessed-data\sub-1448-epo.fif ...
    Found the data of interest:
        t =       0.00 ...   29999.00 ms
        0 CTF compensation matrices available
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
[[0.00110139 0.00079648]
 [0.00123913 0.00097296]
 [0.00124788 0.00088828]
 [0.00121551 0.00082727]
 [0.00114333 0.00093048]
 [0.00123237 0.00104842]
 [0.00128718 0.00116537]
 [0.00135539 0.00129346]
 [0.00120938 0.00128798]
 [0.00067547 0.00074423]
 [0.00107708 0.00114058]
 [0.00127037 0.00112059]
 [0.00148682 0.00090822]
 [0.00163028 0.00118488]
 [0.00144278 0.00164568]
 [0.00145389 0.00159155]
 [0.0013538  0.00145426]
 [0.00130823 0.00125116]
 [0.00192007 0.00099788]
 [0.00146932 0.00087341]
 [0.00180944 0.00116226]
 [0.00206236 0.0011232 ]
 [0.00222609 0.0011786 ]
 [0.001895

In [9]:
print(eeg_power_band(evoked_channels, FREQ_BANDS))

    Using multitaper spectrum estimation with 7 DPSS windows
[[0.00092005 0.00077759]
 [0.00109291 0.00093682]
 [0.00120035 0.00102516]
 [0.00132813 0.00099471]
 [0.00124309 0.00096623]
 [0.0011532  0.00103688]
 [0.00099958 0.00090315]
 [0.00086299 0.00080739]
 [0.00148159 0.00109004]
 [0.00137345 0.00116439]
 [0.00150367 0.0015096 ]
 [0.0014899  0.00135647]
 [0.00158252 0.00105397]
 [0.00136427 0.00111248]
 [0.00142264 0.00148144]
 [0.00139611 0.0014523 ]
 [0.00129363 0.00118454]
 [0.00129459 0.001049  ]
 [0.00186947 0.00198942]
 [0.00197639 0.00189073]
 [0.00195614 0.00186111]
 [0.00188523 0.00175622]
 [0.00141409 0.00183366]
 [0.00136986 0.00210683]
 [0.00136843 0.00213805]
 [0.00136698 0.00233497]
 [0.00148931 0.00272127]
 [0.00106349 0.00324799]]


In [10]:
channels = ['F7', 'F5', 'F3', 'F1', 'F2', 'F4', 'F6', 'F8', 'P7', 'P5', 'P3', 'P1', 'P2', 'P4', 'P6', 'P8', 'O1', 'O2', 'C5', 'C3', 'C1', 'C2', 'C4', 'C6', 'T9', 'T7', 'T8', 'T10']
theta_columns = [f'{channel}_theta' for channel in channels]
alpha_columns = [f'{channel}_alpha' for channel in channels]
columns = theta_columns + alpha_columns

In [11]:
this_df_eeg = pd.DataFrame(columns=["participant_id"]+columns)
for sub_id in df['participant_id'].values:
    try:
        mat_epochs_readed = mne.read_epochs(f"{dir_with_data}{sub_id}-epo.fif")
        evoked = mat_epochs_readed.average()
        evoked_channels = evoked.pick_channels(channels)
        features = eeg_power_band(evoked_channels, FREQ_BANDS)
        features_flatten = features.flatten()
        feat_0 = {column: [feature[0]] for column, feature in zip(theta_columns, features)}
        feat_1 = {column: [feature[1]] for column, feature in zip(alpha_columns, features)}
        name = {"participant_id": [sub_id]}
        print('--------------------------------------------------------------------------------')
        print(feat_0)
        print(feat_1)
        print(name)
        d = {**name, **feat_0, **feat_1}
        new_frame = pd.DataFrame(d)
        print(this_df_eeg)
        print(new_frame)
        this_df_eeg = pd.concat([this_df_eeg, new_frame])
    except:
        print("file for "+sub_id+" doesn't exist")

Reading d:\Nina\Dokumenty\Studia\Kogni\magisterka\I semestr\machine learning\eggML-project\preprocessed-data\sub-1448-epo.fif ...
    Found the data of interest:
        t =       0.00 ...   29999.00 ms
        0 CTF compensation matrices available
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
--------------------------------------------------------------------------------
{'F7_theta': [0.001101393767662747], 'F5_theta': [0.001239126195576545], 'F3_theta': [0.0012478751857910176], 'F1_theta': [0.0012155145046943258], 'F2_theta': [0.0011433336182643577], 'F4_theta': [0.0012323702929286746], 'F6_theta': [0.0012871786427322304], 'F8_theta': [0.0013553855559643956], 'P7_theta': [0.0012093840626148313], 'P5_theta': [0.0006754672263623476], 'P3_theta': [0.0010770810046705423], 'P1_theta': [0.0012703732412993286], 'P2_theta': [0.0014868150367305243], 'P4_theta': [0.001630282

In [12]:
this_df_eeg

,participant_id,F7_theta,F5_theta,F3_theta,F1_theta,F2_theta,F4_theta,F6_theta,F8_theta,P7_theta,...,C5_alpha,C3_alpha,C1_alpha,C2_alpha,C4_alpha,C6_alpha,T9_alpha,T7_alpha,T8_alpha,T10_alpha
0,sub-1448,0.001101,0.001239,0.001248,0.001216,0.001143,0.001232,0.001287,0.001355,0.001209,...,0.000998,0.000873,0.001162,0.001123,0.001179,0.001233,0.001201,0.001202,0.001440,0.001244
0,sub-1824,0.000920,0.000829,0.000997,0.001159,0.001138,0.001094,0.001185,0.001484,0.001082,...,0.002420,0.002401,0.002397,0.002125,0.001925,0.002051,0.002102,0.001935,0.001940,0.001969
0,sub-1971,0.000762,0.000852,0.000856,0.000930,0.000879,0.000814,0.000809,0.000762,0.000775,...,0.000475,0.000477,0.000476,0.000473,0.000537,0.000526,0.000621,0.000626,0.000482,0.000670
0,sub-1989,0.001951,0.001985,0.002088,0.002234,0.002151,0.001946,0.001890,0.001877,0.001277,...,0.003381,0.003541,0.003570,0.003523,0.003481,0.003330,0.003351,0.003329,0.002989,0.003527
0,sub-1990,0.001174,0.001137,0.001207,0.001183,0.001220,0.001281,0.001295,0.001259,0.001393,...,0.001171,0.001073,0.001018,0.000992,0.000953,0.001296,0.001453,0.001827,0.001536,0.001516
0,sub-1998,0.001692,0.001978,0.002296,0.002537,0.002500,0.002273,0.001960,0.001696,0.003731,...,0.001956,0.001952,0.001558,0.001429,0.001428,0.001617,0.001677,0.001777,0.002015,0.001651
0,sub-2009,0.000964,0.000914,0.001204,0.001209,0.001273,0.001236,0.000923,0.001014,0.000726,...,0.001120,0.001199,0.001224,0.001433,0.001613,0.001640,0.001487,0.001649,0.001411,0.001726
0,sub-2015,0.001190,0.001389,0.001905,0.002145,0.002113,0.001710,0.001250,0.001147,0.001332,...,0.003141,0.003048,0.002970,0.002940,0.003097,0.003177,0.003725,0.004980,0.004524,0.005186
0,sub-2017,0.001463,0.001568,0.001531,0.001543,0.001585,0.001609,0.001592,0.001543,0.001185,...,0.000787,0.000812,0.000897,0.000928,0.000829,0.000909,0.000936,0.001027,0.000830,0.000857
0,sub-2018,0.001192,0.001651,0.001704,0.001788,0.001653,0.001520,0.001207,0.000920,0.001002,...,0.001594,0.001686,0.001508,0.001365,0.001415,0.001456,0.001451,0.001444,0.001485,0.001495


In [13]:
this_df_eeg['type'] = this_df_eeg['participant_id'].map(df.set_index('participant_id')['type'])

In [14]:
this_df_eeg

,participant_id,F7_theta,F5_theta,F3_theta,F1_theta,F2_theta,F4_theta,F6_theta,F8_theta,P7_theta,...,C3_alpha,C1_alpha,C2_alpha,C4_alpha,C6_alpha,T9_alpha,T7_alpha,T8_alpha,T10_alpha,type
0,sub-1448,0.001101,0.001239,0.001248,0.001216,0.001143,0.001232,0.001287,0.001355,0.001209,...,0.000873,0.001162,0.001123,0.001179,0.001233,0.001201,0.001202,0.001440,0.001244,Control
0,sub-1824,0.000920,0.000829,0.000997,0.001159,0.001138,0.001094,0.001185,0.001484,0.001082,...,0.002401,0.002397,0.002125,0.001925,0.002051,0.002102,0.001935,0.001940,0.001969,Psychosis
0,sub-1971,0.000762,0.000852,0.000856,0.000930,0.000879,0.000814,0.000809,0.000762,0.000775,...,0.000477,0.000476,0.000473,0.000537,0.000526,0.000621,0.000626,0.000482,0.000670,Control
0,sub-1989,0.001951,0.001985,0.002088,0.002234,0.002151,0.001946,0.001890,0.001877,0.001277,...,0.003541,0.003570,0.003523,0.003481,0.003330,0.003351,0.003329,0.002989,0.003527,Control
0,sub-1990,0.001174,0.001137,0.001207,0.001183,0.001220,0.001281,0.001295,0.001259,0.001393,...,0.001073,0.001018,0.000992,0.000953,0.001296,0.001453,0.001827,0.001536,0.001516,Psychosis
0,sub-1998,0.001692,0.001978,0.002296,0.002537,0.002500,0.002273,0.001960,0.001696,0.003731,...,0.001952,0.001558,0.001429,0.001428,0.001617,0.001677,0.001777,0.002015,0.001651,Psychosis
0,sub-2009,0.000964,0.000914,0.001204,0.001209,0.001273,0.001236,0.000923,0.001014,0.000726,...,0.001199,0.001224,0.001433,0.001613,0.001640,0.001487,0.001649,0.001411,0.001726,Control
0,sub-2015,0.001190,0.001389,0.001905,0.002145,0.002113,0.001710,0.001250,0.001147,0.001332,...,0.003048,0.002970,0.002940,0.003097,0.003177,0.003725,0.004980,0.004524,0.005186,Psychosis
0,sub-2017,0.001463,0.001568,0.001531,0.001543,0.001585,0.001609,0.001592,0.001543,0.001185,...,0.000812,0.000897,0.000928,0.000829,0.000909,0.000936,0.001027,0.000830,0.000857,Control
0,sub-2018,0.001192,0.001651,0.001704,0.001788,0.001653,0.001520,0.001207,0.000920,0.001002,...,0.001686,0.001508,0.001365,0.001415,0.001456,0.001451,0.001444,0.001485,0.001495,Psychosis


In [74]:
#create new data frame to work on 
new_df = this_df_eeg

# select the theta and alpha columns for channels starting with "F"
f_theta_columns = ['F7_theta', 'F5_theta', 'F3_theta', 'F1_theta', 'F2_theta', 'F4_theta', 'F6_theta', 'F8_theta']
f_alpha_columns = ['F7_alpha', 'F5_alpha', 'F3_alpha', 'F1_alpha', 'F2_alpha', 'F4_alpha', 'F6_alpha', 'F8_alpha']
p_theta_columns = ['P1_theta', 'P2_theta', 'P3_theta', 'P4_theta', 'P5_theta', 'P6_theta', 'P7_theta', 'P8_theta']
p_alpha_columns = ['P1_alpha', 'P2_alpha', 'P3_alpha', 'P4_alpha', 'P5_alpha', 'P6_alpha', 'P7_alpha', 'P8_alpha']
o_theta_columns = ['O1_theta', 'O2_theta']
o_alpha_columns = ['O1_alpha', 'O2_alpha']
c_theta_columns = ['C1_theta', 'C2_theta', 'C3_theta', 'C4_theta', 'C5_theta', 'C6_theta']
c_alpha_columns = ['C1_alpha', 'C2_alpha', 'C3_alpha', 'C4_alpha', 'C5_alpha', 'C6_alpha']
t_theta_columns = ['T7_theta', 'T8_theta', 'T9_theta', 'T10_theta']
t_alpha_columns = ['T7_alpha', 'T8_alpha', 'T9_alpha', 'T10_alpha']

# calculate the row-wise mean for the selected columns and rename the resulting columns
new_df['F_average_theta'] = new_df[f_theta_columns].mean(axis=1)
new_df['F_average_alpha'] = new_df[f_alpha_columns].mean(axis=1)
new_df['P_average_theta'] = new_df[p_theta_columns].mean(axis=1)
new_df['P_average_alpha'] = new_df[p_alpha_columns].mean(axis=1)
new_df['o_average_theta'] = new_df[o_theta_columns].mean(axis=1)
new_df['o_average_alpha'] = new_df[o_alpha_columns].mean(axis=1)
new_df['c_average_theta'] = new_df[c_theta_columns].mean(axis=1)
new_df['c_average_alpha'] = new_df[c_alpha_columns].mean(axis=1)
new_df['t_average_theta'] = new_df[t_theta_columns].mean(axis=1)
new_df['t_average_alpha'] = new_df[t_alpha_columns].mean(axis=1)
columns_to_keep = ['type', 'participant_id', 'P_average_theta', 'P_average_alpha', 'F_average_theta', 'F_average_alpha', 'o_average_theta', 'o_average_alpha', 'c_average_theta', 'c_average_alpha', 't_average_theta', 't_average_alpha']

new_df = new_df.drop(columns=[col for col in new_df.columns if col not in columns_to_keep])

print(new_df)

  participant_id       type  F_average_theta  F_average_alpha  \
0       sub-1448    Control         0.001228         0.000990   
0       sub-1824  Psychosis         0.001101         0.001007   
0       sub-1971    Control         0.000833         0.000492   
0       sub-1989    Control         0.002015         0.002209   
0       sub-1990  Psychosis         0.001219         0.001055   
0       sub-1998  Psychosis         0.002117         0.000978   
0       sub-2009    Control         0.001092         0.000983   
0       sub-2015  Psychosis         0.001606         0.001902   
0       sub-2017    Control         0.001554         0.000895   
0       sub-2018  Psychosis         0.001454         0.000886   
0       sub-2028  Psychosis         0.001173         0.001809   
0       sub-2038    Control         0.000894         0.000999   
0       sub-2041  Psychosis         0.001146         0.002443   
0       sub-2044    Control         0.001196         0.000861   
0       sub-2067  Psychos

In [75]:
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, balanced_accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR


# load data
data = new_df

# drop the participant_id column
data = data.drop(columns=['participant_id'])
# replace 'Psychosis' with 1 and 'Control' with 0 in the 'type' column
data['type'] = data['type'].replace({'Psychosis': 1, 'Control': 0})

# separate the target variable from the features
X = data.drop(columns=['type'])
y = data['type']
print(data)

# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# define the parameter grid to search over
param_grid = {'C': [0.1, 1, 10, 100], 'epsilon': [0.1, 0.2, 0.5, 1], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}

# create a Support Vector Regression model
model = SVR()

# perform a grid search to find the best parameters
grid_search = GridSearchCV(model, param_grid, cv=5, scoring=['accuracy', 'roc_auc', 'precision', 'recall', 'balanced_accuracy'], refit='balanced_accuracy')
grid_search.fit(X_train, y_train)


   type  F_average_theta  F_average_alpha  P_average_theta  P_average_alpha  \
0     0         0.001228         0.000990         0.001281         0.001203   
0     1         0.001101         0.001007         0.001248         0.002020   
0     0         0.000833         0.000492         0.000753         0.000498   
0     0         0.002015         0.002209         0.002111         0.003382   
0     1         0.001219         0.001055         0.001277         0.001016   
0     1         0.002117         0.000978         0.003518         0.001390   
0     0         0.001092         0.000983         0.001133         0.001129   
0     1         0.001606         0.001902         0.001685         0.002100   
0     0         0.001554         0.000895         0.001421         0.000979   
0     1         0.001454         0.000886         0.001272         0.001028   
0     1         0.001173         0.001809         0.000904         0.003230   
0     0         0.000894         0.000999         0.

GridSearchCV(cv=5, estimator=SVR(),
             param_grid={'C': [0.1, 1, 10, 100], 'epsilon': [0.1, 0.2, 0.5, 1],
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
             refit='balanced_accuracy',
             scoring=['accuracy', 'roc_auc', 'precision', 'recall',
                      'balanced_accuracy'])

In [76]:
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# make predictions on the testing data using the best model
y_pred = best_model.predict(X_test)

# convert the predicted values to binary format
y_pred_bin = [1 if x > 0.5 else 0 for x in y_pred]

# convert the actual values to binary format
y_test_bin = [1 if x > 0.5 else 0 for x in y_test]

# evaluate the performance of the best model using different metrics
accuracy = accuracy_score(y_test_bin, y_pred_bin)
roc_auc = roc_auc_score(y_test_bin, y_pred_bin)
precision = precision_score(y_test_bin, y_pred_bin)
recall = recall_score(y_test_bin, y_pred_bin)
balanced_accuracy = balanced_accuracy_score(y_test_bin, y_pred_bin)

# print out the best parameters and the evaluation metrics for the best model
print("Best parameters:", best_params)
print("Accuracy:", accuracy)
print("ROC AUC:", roc_auc)
print("Precision:", precision)
print("Recall:", recall)
print("Balanced accuracy:", balanced_accuracy)

Best parameters: {'C': 0.1, 'epsilon': 0.1, 'kernel': 'linear'}
Accuracy: 0.7
ROC AUC: 0.5
Precision: 0.7
Recall: 1.0
Balanced accuracy: 0.5
